In [31]:
import numpy as np
import pandas as pd
from random import randint

In [246]:
alphabet = 'ABC'

In [227]:
def generate_matrix(alphabet):
    '''Генерация случайной матрицы весов по заданному алфавиту'''
    if '_' not in alphabet:
        alphabet += '_'
    ndim = len(alphabet)

    weights = np.zeros((ndim, ndim), dtype=np.int8)
 
    for i, symb1 in enumerate(alphabet):
        tmp = []
        for j, symb2 in enumerate(alphabet):
            if symb1 != '_' and symb2 != '_':
                if symb1 == symb2:
                    weights[i][j] = 2
                else:
                    weights[i][j] = weights[j][i] = randint(-2, 2)
            else:
                weights[i][j] = weights[j][i] = randint(-2, 0)

    res = pd.DataFrame(weights)
    res = res.transpose()
    res.insert(loc=0, column='*', value=list(alphabet))
    res = res.set_index('*')
    res.columns = list(alphabet)
    
    return res

In [228]:
weights = generate_matrix(alphabet)
weights

,A,B,C,_
*,,,,
A,2,-2,2,-1
B,-2,2,2,-2
C,2,2,2,-1
_,-1,-2,-1,0


In [248]:
def align(s1, s2):
    n, m = len(s1), len(s2)
    
    nlines = n + 1
    ncols = m + 1
    
    dp = [ [float('-inf') for _ in range(ncols)] for __ in range(nlines)] 
    
    dp[0][0] = 0
    for i, symb1 in enumerate(s1):
        dp[i+1][0] = dp[i][0] + weights[symb1]['_']
        
    for i, symb2 in enumerate(s2):
        dp[0][i+1] = dp[0][i] + weights[symb2]['_']
   
    """for line in dp:
        print(line)"""
        
    for i, symb1 in enumerate(s1):
        for j, symb2 in enumerate(s2):
            
            diag = dp[i][j] + weights[symb1][symb2]
            up = dp[i][j+1] + weights['_'][symb2]
            left = dp[i+1][j] + weights[symb1]['_']
            
            best = max(diag, up, left)
            
            dp[i+1][j+1] = best
    
    """print('****')
    for line in dp:
        print(line)"""
    
    cur_i, cur_j = n, m
    
    line1, line2 = [], []     # инвертированные выравненные последовательности
    
    diag = dp[cur_i - 1][cur_j - 1]
    up = dp[cur_i - 1][cur_j]
    left = dp[cur_i][cur_j - 1]
    
    # print(matrix[n][m])
    
    best = max(diag, up, left)
    if best == diag:
        line1.append(s1[cur_i - 1])
        line2.append(s2[cur_j - 1])
        cur_i -= 1
        cur_j -= 1
    elif best == up:
        line1.append(s1[cur_i - 1])
        line2.append('_')
        cur_i -= 1
    else:
        line1.append('_')
        line2.append(s2[cur_j - 1])
        cur_j -= 1
    
    while cur_i and cur_j:
        
        diag = dp[cur_i - 1][cur_j - 1]
        up = dp[cur_i - 1][cur_j]
        left = dp[cur_i][cur_j - 1]

        best = max(diag, up, left)
        if best == diag:
            line1.append(s1[cur_i - 1])
            line2.append(s2[cur_j - 1])
            cur_i -= 1
            cur_j -= 1
        elif best == up:
            line1.append(s1[cur_i - 1])
            line2.append('_')
            cur_i -= 1
        else:
            line1.append('_')
            line2.append(s2[cur_j - 1])
            cur_j -= 1
    line1 = line1[::-1]
    line2 = line2[::-1]
    
    print(*line1, sep='')
    print(*line2, sep='')

In [249]:
s1, s2 = 'ABACABA', 'ABBA'
align(s1, s2)

ABACABA
AB_BA__


In [253]:
# теперь изменим матрицу весов:
weights['B']['C'] = weights['C']['B'] = -4

In [254]:
align(s1, s2)

AB_ACABA
ABBA____
